In [1]:
import pandas as pd

df = pd.read_csv("C:\\MyWork\\MyLearning\\ML\\Files\\DataSet\\labeledTrainData.tsv",sep='\t', quoting=3)

print(df.shape)

df.head()

(25000, 3)


,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [2]:
import re

def clean_review(text):
    # Strip HTML tags
    text = re.sub('<[^<]+?>', ' ', text)
    
    # Strip escaped quotes
    text = text.replace('\\"', '')
    
    # Strip quotes
    text = text.replace('"', '')
    
    return text

In [3]:
df['cleaned_review'] = df['review'].apply(clean_review)

In [4]:
df.head()

,id,sentiment,review,cleaned_review
0,"""5814_8""",1,"""With all this stuff going down at the moment ...",With all this stuff going down at the moment w...
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ...",The Classic War of the Worlds by Timothy Hines...
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell...",The film starts with a manager (Nicholas Bell)...
3,"""3630_4""",0,"""It must be assumed that those who praised thi...",It must be assumed that those who praised this...
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ...",Superbly trashy and wondrously unpretentious 8...


In [5]:
from sklearn.model_selection import train_test_split

# Train Test Split Data
X_train, X_test, y_train, y_test = train_test_split(df['cleaned_review'], df['sentiment'], test_size=0.2)

In [6]:
# One Hot Encoding of the Train Data

from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

vectorizer = CountVectorizer(binary=True, stop_words=stopwords.words('english'),
                             lowercase=True, min_df=3, max_df=0.9, max_features=5000)

X_train_onehot = vectorizer.fit_transform(X_train)

In [7]:
X_train_onehot.shape

(20000, 5000)

In [8]:
X_train_onehot.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [62]:
# Building Model

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Initialize the Model
model = Sequential()

# Adding Model Layers
model.add(Dense(500,activation='relu',input_dim=5000,))
model.add(Dense(1,activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

model.summary()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 500)               2500500   
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 501       
Total params: 2,501,001
Trainable params: 2,501,001
Non-trainable params: 0
_________________________________________________________________


In [65]:
# Train the model
model.fit(X_train_onehot[:-100],y_train[:-100],epochs=2,batch_size=128,validation_data=(X_train_onehot[-100:], y_train[-100:]))

Train on 19900 samples, validate on 100 samples
Epoch 1/2
19900/19900 [==============================] - 4s 196us/sample - loss: 0.3491 - acc: 0.8541 - val_loss: 0.2503 - val_acc: 0.8900
Epoch 2/2
19900/19900 [==============================] - 3s 160us/sample - loss: 0.1844 - acc: 0.9311 - val_loss: 0.2275 - val_acc: 0.8900


In [66]:
scores = model.evaluate(vectorizer.transform(X_test), y_test, verbose=1)
print("Accuracy:", scores[1])  # Accuracy: 0.875

5000/5000 [==============================] - 1s 100us/sample - loss: 0.3125 - acc: 0.8758
Accuracy: 0.8758
